# EDS-TEM quantification of core shell nanoparticles

Using machine learning methods, such as independent component analysis (ICA), the composition of embedded nanostructures, such as core-shell nanoparticles, can be accurately measured as demonstrated by D. Roussow et al., Nano Letters, 2015 (see the [full article](https://www.repository.cam.ac.uk/bitstream/handle/1810/248102/Roussouw%20et%20al%202015%20Nano%20Letters.pdf?sequence=1)). Using the same data, this notebook reproduces the main results of this article.

This notebook requires:

HyperSpy 0.8.5 and all dependencies

## Authors

Original:

13/04/15 Pierre Burdet - Developed for HyperSpy workshop at University of Cambridge

Versions:

29/05/2016 Duncan Johnstone. Update the syntax for HyperSpy 0.8.5 (Python 3 compatibility)

06/08/2015 Francisco de la Peña. Update the syntax for HyperSpy 0.8.1

## <a id='top'></a> Contents

1. <a href='dat'> Specimen & Data</a>
2. <a href='#loa'> Loading</a>
3. <a href='#bss'> Blind source separation of core/shell nanoparticles</a>
4. <a href='#bare'> Representative spectrum from bare cores</a>
5. <a href='#com'> Comparison and quantification</a>
6. <a href='#fur'> Going futher: Isolating the nanoparticles</a>

# <a id='dat'></a> 1. Specimen & Data

The sample and the data used in this tutorial are described in 
D. Roussow, et al., Nano Letters, In Press (2015) (see the [full article](https://www.repository.cam.ac.uk/bitstream/handle/1810/248102/Roussouw%20et%20al%202015%20Nano%20Letters.pdf?sequence=1)).

FePt@Fe$_3$O$_4$ core-shell nanoparticles are investigated with an EDS/TEM experiment (FEI Osiris TEM, 4 EDS detectors). The composition of the core can be measured with ICA (see figure 1c). To prove the accuracy of the results, measurments on bare FePt bimetallic nanoparticles from a synthesis prior to the shell addition step are used.

<img src="images/core_shell.png" style="height:350px;">
Figure 1: (a) A spectrum image obtained from a cluster of core-shell nanoparticles. (b) The nanoparticles are comprised of a bi-metallic Pt/Fe core surrounded by an iron oxide shell on a carbon support. (c) ICA decomposes the mixed EDX signals into components representing the core (IC#0), shell (IC#1) and support (IC#2).

In [1]:
#Download the data (1MB)
from urllib.request import urlretrieve, urlopen
fnames= ['bare_core.hdf5', 'core_shell.hdf5']   
url = 'http://cook.msm.cam.ac.uk//~hyperspy//EDS_tutorial//'
files = [urlretrieve(url + fname, fname) for fname in fnames]

# <a id='loa'></a> 2. Loading

Import HyperSpy, numpy and matplotlib libraries

In [2]:
%matplotlib qt4
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt


Writing the config file


For a nice on-line display (optional)

In [ ]:
%matplotlib inline

<a href='#top'> Table of contents</a>

Load the spectrum images of the bare seeds and the core shell nanoparticles.

In [3]:
c = hs.load('bare_core.hdf5')
cs = hs.load('core_shell.hdf5')

/home/dnj23/hyperspy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)


In [4]:
c.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 35.0
│       │       └── energy_resolution_MnKa = 133.31
│       ├── acquisition_mode = STEM
│       ├── beam_energy = 200.0
│       ├── microscope = Microscope TecnaiOsiris 200 kV D658 AnalyticalTwin
│       └── tilt_stage = 0.0
├── General
│   ├── date = 14.10.2014
│   └── title = Bare core
├── Sample
│   ├── elements = array(['Fe', 'Pt'], 
      dtype='<U2')
│   └── xray_lines = array(['Fe_Ka', 'Pt_La'], 
      dtype='<U5')
└── Signal
    ├── binned = True
    ├── record_by = spectrum
    ├── signal_origin = 
    └── signal_type = EDS_TEM

Plot the intensity of Fe K${\alpha}$ and Pt L${\alpha}$.

In [5]:
axes = hs.plot.plot_images(c.get_lines_intensity() + cs.get_lines_intensity(),
                           scalebar='all', axes_decor=None, per_row=2, cmap='RdBu')

/home/dnj23/hyperspy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)
/home/dnj23/hyperspy/hyperspy/signals.py:126: VisibleDeprecationWarning: The Signal class will be deprecated from version 1.0.0 and replaced with BaseSignal
  VisibleDeprecationWarning)


## <a id='bss'></a> 3. Blind source separation of core/shell nanoparticles

<a href='#top'> Table of contents</a>

Apply blind source separation (ICA) to obtain a factor (spectrum) corresponding to the core.

In [6]:
cs.change_dtype('float')
cs.decomposition()

/home/dnj23/hyperspy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)
/home/dnj23/hyperspy/hyperspy/signals.py:126: VisibleDeprecationWarning: The Signal class will be deprecated from version 1.0.0 and replaced with BaseSignal
  VisibleDeprecationWarning)



Scaling the data to normalize the (presumably) Poissonian noise

Performing decomposition analysis
Undoing data pre-treatments


In [7]:
ax = cs.plot_explained_variance_ratio()

ICA on the three first components.

In [8]:
cs.blind_source_separation(3)

IC 0 reversed
IC 2 reversed


/home/dnj23/hyperspy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)


In [9]:
axes = hs.plot.plot_images(cs.get_bss_loadings(), axes_decor=None, cmap='RdBu')

/home/dnj23/hyperspy/hyperspy/signals.py:126: VisibleDeprecationWarning: The Signal class will be deprecated from version 1.0.0 and replaced with BaseSignal
  VisibleDeprecationWarning)


In [10]:
axes = hs.plot.plot_spectra(cs.get_bss_factors())

/home/dnj23/hyperspy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)
/home/dnj23/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:892: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/home/dnj23/hyperspy/hyperspy/signal.py:1331: VisibleDeprecationWarning: Indexing the `Signal` class is deprecated and will be removed in Hyperspy 1.0. Please use `.isig` and/or `.inav` instead.
  VisibleDeprecationWarning)


The first component corresponds to the core.

In [11]:
s_bss = cs.get_bss_factors().inav[0]

/home/dnj23/hyperspy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)


## <a id='bare'></a> 4. Representative spectrum from bare cores

<a href='#top'> Table of contents</a>

To obtain a representative spectrum of the bare nanoparticles, the low intensity of Pt L${\alpha}$ is masked.

In [12]:
pt_la = c.get_lines_intensity(['Pt_La'])[0]
mask = pt_la > 6

/home/dnj23/hyperspy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)
/home/dnj23/hyperspy/hyperspy/signals.py:126: VisibleDeprecationWarning: The Signal class will be deprecated from version 1.0.0 and replaced with BaseSignal
  VisibleDeprecationWarning)


In [13]:
axes = hs.plot.plot_images([mask, pt_la * mask], axes_decor=None, colorbar=None,
                           label=['Mask', 'Pt L${\\alpha}$ intensity'], cmap='RdBu')

To apply the mask, the navigation dimensions of the mask must be manipulated to match the navigation dimensions of the EDS spectrum image. This is achived crudely via first generating a mask using the built in vacuum_mask() method and then overwriting the data with the mask generated above.

In [14]:
c_mask = c.vacuum_mask()
c_mask.data = mask.data

/home/dnj23/hyperspy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)
/home/dnj23/hyperspy/hyperspy/signals.py:126: VisibleDeprecationWarning: The Signal class will be deprecated from version 1.0.0 and replaced with BaseSignal
  VisibleDeprecationWarning)


The sum over the particles is used as a bare core spectrum.

In [15]:
s_bare = (c * c_mask).sum(0).sum(0)

/home/dnj23/hyperspy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)


## <a id='com'></a> 5. Comparison and quantification

<a href='#top'> Table of contents</a>

Stack together the spectrum of bare particles and the first ICA component.

In [16]:
s_bare.change_dtype('float')
s = hs.stack([s_bare, s_bss], new_axis_name='Bare or BSS')
s.metadata.General.title = 'Bare or BSS'

/home/dnj23/hyperspy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)


In [17]:
axes = hs.plot.plot_spectra(s, style='mosaic', legend=['Bare particles', 'BSS #0'])

/home/dnj23/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:892: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/home/dnj23/hyperspy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)


X-ray intensities measurement with background subtraction.

In [18]:
w = s.estimate_background_windows()

In [19]:
s.plot(background_windows=w)

/home/dnj23/hyperspy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)
/home/dnj23/hyperspy/hyperspy/signals.py:126: VisibleDeprecationWarning: The Signal class will be deprecated from version 1.0.0 and replaced with BaseSignal
  VisibleDeprecationWarning)
/home/dnj23/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:892: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Refinement of the windows position.

In [20]:
w

array([[ 5.99000266,  6.12796844,  6.67983156,  6.81779734],
       [ 8.95253846,  9.11572564,  9.76847436,  9.93166154]])

In [21]:
w[1, 0] = 8.44
w[1, 1] = 8.65

In [23]:
s.plot(background_windows=w, navigator='slider')

/home/dnj23/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:892: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [22]:
sI = s.get_lines_intensity(background_windows=w)

/home/dnj23/hyperspy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)
/home/dnj23/hyperspy/hyperspy/signals.py:126: VisibleDeprecationWarning: The Signal class will be deprecated from version 1.0.0 and replaced with BaseSignal
  VisibleDeprecationWarning)


Set up the kfactors for Fe K${\alpha}$ and Pt L${\alpha}$.

In [23]:
#From Brucker software (Esprit)
kfactors = [1.450226, 5.075602]

Quantify with Cliff Lorimer.

In [25]:
composition = s.quantification(intensities=sI, kfactors=kfactors,
                 plot_result=True, composition_units='atomic')

/home/dnj23/hyperspy/hyperspy/signals.py:54: VisibleDeprecationWarning: The Spectrum class will be deprecated from version 1.0.0 and replaced with Signal1D
  VisibleDeprecationWarning)
/home/dnj23/hyperspy/hyperspy/signals.py:126: VisibleDeprecationWarning: The Signal class will be deprecated from version 1.0.0 and replaced with BaseSignal
  VisibleDeprecationWarning)


## <a id='fur'></a> 6. Going further

<a href='#top'> Table of contents</a>

Further image processing with [scikit-image](http://scikit-image.org/) and [scipy](http://www.scipy.org/). Apply a watershed transformation to isolate the nanoparticles.

- Transform the mask into a distance map.
- Find local maxima.
- Apply the watershed to the distance map using the local maximum as seed (markers).

Adapted from this scikit-image [example](http://scikit-image.org/docs/dev/auto_examples/plot_watershed.html).

In [26]:
from scipy.ndimage import distance_transform_edt, label
from skimage.morphology import watershed
from skimage.feature import peak_local_max

In [27]:
distance = distance_transform_edt(mask.data)
local_maxi = peak_local_max(distance, indices=False,
                            min_distance=2, labels=mask.data)
labels = watershed(-distance, markers=label(local_maxi)[0],
                   mask=mask.data)

In [28]:
axes = hs.plot.plot_images(
    [pt_la, mask, hs.signals.Image(distance), hs.signals.Image(labels)],
    axes_decor='off', per_row=2, colorbar=None, cmap='RdYlBu_r',
    label=['Pt L${\\alpha}$ intensity', 'Mask',
           'Distances', 'Separated particles'])

/home/dnj23/hyperspy/hyperspy/signals.py:83: VisibleDeprecationWarning: The Image class will be deprecated from version 1.0.0 and replaced with Signal2D
  VisibleDeprecationWarning)
